# NB Regression with PyTorch

Negative binomial regression is in fact a 1-layer neural network with a special loss function. Here we increase the number of layers so that the mean $\mu$ of the negative binomial distribution is no longer approximated by a purely linear combination of the features. At the same time, we also regress the dispersion $\alpha$ which determines the variance ($\mu+\alpha\mu²$).

In [24]:
from NBPyTorch import NBNet, PoNet, NBNLLLoss
import pandas as pd
from sklearn.model_selection import train_test_split
import torch.optim as optim
import torch
import numpy as np
import scipy as sc
from util import MyUtil

%load_ext autoreload
%autoreload 2

util=MyUtil()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
def to_numpy(muA,alphaA,muB,alphaB):
    return muA.data.numpy(),alphaA.data.numpy(),muB.data.numpy(),alphaB.data.numpy()

In [26]:
# data/final.csv: past tournaments
# data/final_w_18: updated with WC 2018 results
data = pd.read_csv(filepath_or_buffer="data/final.csv",delimiter=";",index_col=False).round(2)

# impute missing past values with 0
data.fillna({"past_resultA":0,"past_resultB":0},inplace=True)
# drop id columns
data.drop(["gameid","teamidA","teamidB"],axis=1,inplace=True)

# Encode gametype
rounds = {
    "Finale": 1,
    "Spiel um Platz Drei": 2,
    "Halbfinale": 3,
    "Viertelfinale": 4,
    "Achtelfinale": 5,
    "Gruppenphase": 6
}
def map_to_round(x):
    if x.startswith("Gruppe"):
        return rounds["Gruppenphase"]
    else:
        return rounds[x]

data["gametype"]=data["gametype"].apply(map_to_round)
data.dtypes

tournament        object
gametype           int64
teamA             object
teamB             object
resultA            int64
resultB            int64
addinfo           object
date              object
teamA_age        float64
teamB_age        float64
teamA_def_val    float64
teamB_def_val    float64
teamA_off_val    float64
teamB_off_val    float64
teamA_frag         int64
teamB_frag         int64
past_resultA     float64
past_resultB     float64
dtype: object

In [27]:
# knockout stage w/o pens
data_ko=data.drop(data[data.gametype == 6].index,axis=0).copy()
data_ko.drop(data_ko[data_ko.addinfo == 'n.E.'].index,axis=0,inplace=True) # drop pens

# group stage 
data_gr=data.drop(data[data.gametype != 6].index,axis=0)

## Knockout stage

In [28]:
data_ko.tail(8)

,tournament,gametype,teamA,teamB,resultA,resultB,addinfo,date,teamA_age,teamB_age,teamA_def_val,teamB_def_val,teamA_off_val,teamB_off_val,teamA_frag,teamB_frag,past_resultA,past_resultB
513,AC13,4,Elfenbeinküste,Nigeria,1,2,NaN,03.02.13,27.83,23.95,3219230.77,2830769.23,9765000.00,5483333.33,19,19,1.00,0.00
514,AC13,4,Burkina Faso,Togo,1,0,n.V.,03.02.13,26.41,25.17,919230.77,620833.33,1294444.44,1577272.73,19,18,0.95,0.95
515,AC13,4,Ghana,Kap Verde,2,0,NaN,02.02.13,24.39,25.48,1313333.33,359615.38,4781250.00,1020000.00,20,20,3.90,0.00
541,GC11,1,USA,Mexiko,2,4,NaN,26.06.11,26.83,26.17,2622727.27,2833333.33,2925000.00,4541666.67,16,16,0.95,2.14
542,GC11,3,USA,Panama,1,0,NaN,23.06.11,27.00,24.56,2850000.00,477272.73,2246428.57,182142.86,16,16,1.05,1.95
543,GC11,3,Honduras,Mexiko,0,2,n.V.,23.06.11,26.56,26.17,1958333.33,2833333.33,750000.00,4541666.67,11,16,0.15,1.00
545,GC11,4,Mexiko,Guatemala,2,1,NaN,19.06.11,26.21,25.15,3187500.00,171428.57,4541666.67,266666.67,13,9,0.11,0.15
546,GC11,4,Jamaika,USA,0,2,NaN,19.06.11,26.93,26.36,415625.00,3550000.00,495833.33,3153571.43,11,13,1.00,1.10


In [29]:
data_train, data_test = train_test_split(data_ko.copy(),test_size=0.2)
col=["gametype","teamA_age","teamB_age","teamA_def_val","teamB_def_val","teamA_off_val","teamB_off_val","teamA_frag","teamB_frag","past_resultA","past_resultB"]

In [30]:
teamA_def_mean = data_train["teamA_def_val"].mean()
teamA_def_std = data_train["teamA_def_val"].std()
teamA_off_mean = data_train["teamA_off_val"].mean()
teamA_off_std = data_train["teamA_off_val"].std()

teamB_def_mean = data_train["teamB_def_val"].mean()
teamB_def_std = data_train["teamB_def_val"].std()
teamB_off_mean = data_train["teamB_off_val"].mean()
teamB_off_std = data_train["teamB_off_val"].std()

teamA_frag_mean = data_train["teamA_frag"].mean()
teamA_frag_std = data_train["teamA_frag"].std()

teamB_frag_mean = data_train["teamB_frag"].mean()
teamB_frag_std = data_train["teamB_frag"].std()

teamA_age_mean = data_train["teamA_age"].mean()
teamA_age_std = data_train["teamA_age"].std()

teamB_age_mean = data_train["teamB_age"].mean()
teamB_age_std = data_train["teamB_age"].std()

In [31]:
# scale train features
data_train["teamA_def_val"]=(data_train["teamA_def_val"]-teamA_def_mean)/teamA_def_std
data_train["teamA_off_val"]=(data_train["teamA_off_val"]-teamA_off_mean)/teamA_off_std
data_train["teamB_def_val"]=(data_train["teamB_def_val"]-teamB_def_mean)/teamB_def_std
data_train["teamB_off_val"]=(data_train["teamB_off_val"]-teamB_off_mean)/teamB_off_std
data_train["teamA_frag"]=(data_train["teamA_frag"]-teamA_frag_mean)/teamA_frag_std
data_train["teamB_frag"]=(data_train["teamB_frag"]-teamB_frag_mean)/teamB_frag_std
data_train["teamA_age"]=(data_train["teamA_age"]-teamA_age_mean)/teamA_age_std
data_train["teamB_age"]=(data_train["teamB_age"]-teamB_age_mean)/teamB_age_std

# scale test features
data_test["teamA_def_val"]=(data_test["teamA_def_val"]-teamA_def_mean)/teamA_def_std
data_test["teamA_off_val"]=(data_test["teamA_off_val"]-teamA_off_mean)/teamA_off_std
data_test["teamB_def_val"]=(data_test["teamB_def_val"]-teamB_def_mean)/teamB_def_std
data_test["teamB_off_val"]=(data_test["teamB_off_val"]-teamB_off_mean)/teamB_off_std
data_test["teamA_frag"]=(data_test["teamA_frag"]-teamA_frag_mean)/teamA_frag_std
data_test["teamB_frag"]=(data_test["teamB_frag"]-teamB_frag_mean)/teamB_frag_std
data_test["teamA_age"]=(data_test["teamA_age"]-teamA_age_mean)/teamA_age_std
data_test["teamB_age"]=(data_test["teamB_age"]-teamB_age_mean)/teamB_age_std

In [32]:
data_train.describe()

,gametype,resultA,resultB,teamA_age,teamB_age,teamA_def_val,teamB_def_val,teamA_off_val,teamB_off_val,teamA_frag,teamB_frag,past_resultA,past_resultB
count,89.000000,89.000000,89.000000,8.900000e+01,8.900000e+01,8.900000e+01,8.900000e+01,8.900000e+01,8.900000e+01,8.900000e+01,8.900000e+01,89.000000,89.000000
mean,3.460674,1.573034,1.191011,-1.947256e-15,-3.253328e-15,7.235161e-17,-2.245395e-17,8.732091e-17,-6.985673e-17,2.295293e-16,-1.521879e-16,1.160337,1.112472
std,1.340422,1.287065,1.251149,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.307872,1.074248
min,1.000000,0.000000,0.000000,-3.150748e+00,-2.124791e+00,-1.021065e+00,-8.597411e-01,-1.132683e+00,-9.157056e-01,-1.956158e+00,-2.202777e+00,0.000000,0.000000
25%,3.000000,1.000000,0.000000,-6.908092e-01,-5.865828e-01,-8.045210e-01,-6.680256e-01,-8.807878e-01,-7.045045e-01,-6.760253e-01,-7.239732e-01,0.050000,0.050000
50%,4.000000,1.000000,1.000000,1.072312e-01,-1.169674e-02,-3.956967e-01,-4.297661e-01,-2.048655e-01,-4.562801e-01,9.205451e-02,1.210578e-01,1.000000,1.000000
75%,5.000000,2.000000,2.000000,7.325000e-01,7.185639e-01,5.070001e-01,3.297609e-01,6.513724e-01,5.383002e-01,6.041077e-01,7.548310e-01,1.900000,1.900000
max,5.000000,6.000000,6.000000,1.843175e+00,1.860567e+00,2.742703e+00,3.288464e+00,2.600330e+00,2.984362e+00,1.884241e+00,1.599862e+00,6.700000,5.750000


In [33]:
X_train = data_train[col].values
y_train = data_train[["resultA","resultB"]].values
X_test = data_test[col].values
y_test = data_test[["resultA","resultB"]].values

In [34]:
X_tr = torch.from_numpy(X_train).float()
y_trA = torch.from_numpy(y_train[:,[0]]).float()
y_trB = torch.from_numpy(y_train[:,[1]]).float()
X_te = torch.from_numpy(X_test).float()

### Negative Binomial distribution

#### Model

In [12]:
crit = NBNLLLoss(eps=1e-3,verbose=False)
def fn_print(module, grad_input, grad_output):
    print("Gradients Input",grad_input)
    print("Gradients Output", grad_output)
    return None

In [13]:
neural = NBNet(len(col),30,20,4,0.5)
optimizer = optim.RMSprop(params=neural.parameters(), lr=1e-3, alpha=0.99, eps=1e-05, weight_decay=0, momentum=0, centered=False)

In [14]:
for epoch in range(200):
    # train
    neural.train()
    muA,alphaA,muB,alphaB = neural(X_tr)
    lossA = crit(muA,alphaA,y_trA)
    lossB = crit(muB,alphaB,y_trB)
    loss = lossA+lossB
    muA,alphaA,muB,alphaB=to_numpy(muA,alphaA,muB,alphaB)
    tend_acc_tr = util.tend_acc_nb(muA,alphaA,muB,alphaB,y_train)
    # update
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # evaluate
    neural.eval()
    muA,alphaA,muB,alphaB = neural(X_te)
    muA,alphaA,muB,alphaB=to_numpy(muA,alphaA,muB,alphaB)
    tend_acc_val = util.tend_acc_nb(muA,alphaA,muB,alphaB,y_test)
    print(epoch,loss.item(),tend_acc_tr,tend_acc_val)


0 3.2290244102478027 0.5168539325842697 0.6086956521739131
1 3.2297208309173584 0.550561797752809 0.6086956521739131
2 3.1443190574645996 0.5842696629213483 0.6521739130434783
3 3.143949031829834 0.5842696629213483 0.6086956521739131
4 3.1881117820739746 0.5955056179775281 0.6521739130434783
5 3.154956817626953 0.5955056179775281 0.6086956521739131
6 3.2091565132141113 0.5393258426966292 0.6086956521739131
7 3.1498215198516846 0.6067415730337079 0.6521739130434783
8 3.1710119247436523 0.5056179775280899 0.6521739130434783
9 3.1463379859924316 0.6292134831460674 0.6521739130434783
10 3.128599166870117 0.6067415730337079 0.6521739130434783
11 3.1305060386657715 0.6853932584269663 0.6521739130434783
12 3.0908002853393555 0.6179775280898876 0.6086956521739131
13 3.110560894012451 0.651685393258427 0.6086956521739131
14 3.1394729614257812 0.550561797752809 0.6521739130434783
15 3.100520610809326 0.6404494382022472 0.6086956521739131
16 3.0905399322509766 0.6067415730337079 0.695652173913043

139 2.9176738262176514 0.6853932584269663 0.6521739130434783
140 2.8875811100006104 0.7752808988764045 0.6521739130434783
141 2.9195761680603027 0.7078651685393258 0.6521739130434783
142 2.9549942016601562 0.6853932584269663 0.6521739130434783
143 2.9578917026519775 0.6179775280898876 0.6521739130434783
144 2.9062752723693848 0.6292134831460674 0.6521739130434783
145 2.9013543128967285 0.7303370786516854 0.6956521739130435
146 2.8675968647003174 0.6629213483146067 0.6956521739130435
147 2.9194488525390625 0.7415730337078652 0.6956521739130435
148 2.945643424987793 0.6966292134831461 0.6956521739130435
149 2.858874797821045 0.7640449438202247 0.6956521739130435
150 2.8615283966064453 0.7078651685393258 0.6521739130434783
151 2.9395923614501953 0.6966292134831461 0.6956521739130435
152 2.868809700012207 0.7640449438202247 0.6521739130434783
153 2.9435105323791504 0.7752808988764045 0.6521739130434783
154 2.958925247192383 0.7303370786516854 0.6521739130434783
155 2.893820285797119 0.6853

#### Validation

In [15]:
neural.eval()
muA,alphaA,muB,alphaB=neural(X_te)
muA,alphaA,muB,alphaB=to_numpy(muA,alphaA,muB,alphaB)

In [17]:
probs = util.calc_nb_probs(muA,alphaA,muB,alphaB)
util.multi_result(y=y_test,y_prob=probs,top_n=1,verbose=True)

wrong prediction for [1 0]
candidates 
 [[1.         1.         0.08803572]]
wrong prediction for [1 2]
candidates 
 [[1.         0.         0.13940216]]
wrong prediction for [2 0]
candidates 
 [[1.         0.         0.11786911]]
wrong prediction for [1 3]
candidates 
 [[1.         0.         0.15063886]]
wrong prediction for [0 2]
candidates 
 [[0.         1.         0.10503696]]
wrong prediction for [1 7]
candidates 
 [[0.         0.         0.22547954]]
wrong prediction for [1 2]
candidates 
 [[2.         0.         0.09954004]]
wrong prediction for [2 1]
candidates 
 [[1.         0.         0.12076676]]
right prediction for [1 0]
candidates 
 [[1.         0.         0.12936794]]
wrong prediction for [2 4]
candidates 
 [[1.         1.         0.10184319]]
wrong prediction for [2 0]
candidates 
 [[1.         1.         0.10150991]]
wrong prediction for [2 3]
candidates 
 [[1.         1.         0.08951705]]
wrong prediction for [2 1]
candidates 
 [[1.         0.         0.11309314]]

0.08695652173913043

From the above results, we can see that the predictions are much more diverse (4 or 5 goals can be predicted). It no longer looks as if the model memorized the most frequent game outcomes. Instead it looks like the goal counts in top-5 predictions really depend on the features.

In [18]:
state = {
    "teamA_def_mean": teamA_def_mean,
    "teamA_def_std": teamA_def_std,
    "teamA_off_mean": teamA_off_mean,
    "teamA_off_std": teamA_off_std,
    "teamB_def_mean": teamB_def_mean,
    "teamB_def_std": teamB_def_std,
    "teamB_off_mean": teamB_off_mean,
    "teamB_off_std": teamB_off_std,
    "teamA_frag_mean": teamA_frag_mean, 
    "teamA_frag_std": teamA_frag_std,
    "teamB_frag_mean": teamB_frag_mean, 
    "teamB_frag_std": teamB_frag_std,
    "teamA_age_mean": teamA_age_mean, 
    "teamA_age_std": teamA_age_std,
    "teamB_age_mean": teamB_age_mean,
    "teamB_age_std": teamB_age_std,
    "state_dict": neural.state_dict(),
}
#torch.save(state, 'model/model_ko8.pth')

### Poisson distribution

In [35]:
crit_pois = torch.nn.PoissonNLLLoss()
neural_pois = PoNet(len(col),30,20,2,0.2)
optimizer_pois = optim.RMSprop(params=neural.parameters(), lr=1e-3, alpha=0.99, eps=1e-05, weight_decay=0, momentum=0, centered=False)

In [37]:
for epoch in range(200):
    # train
    neural_pois.train()
    mu = neural_pois(X_tr)
    lossA_pois = crit_pois(mu[:,[0]],y_trA)
    lossB_pois = crit_pois(mu[:,[1]],y_trB)
    loss_pois = lossA_pois+lossB_pois    
    tend_acc_tr = util.tend_acc_pois(mu.data.numpy(),y_train)
    # evaluate
    neural_pois.eval()
    mu = neural_pois(X_te)
    tend_acc_val = util.tend_acc_pois(mu.data.numpy(),y_test)
    print(epoch, loss_pois.item(),tend_acc_tr,tend_acc_val)
    # update
    optimizer_pois.zero_grad()
    loss_pois.backward()
    optimizer_pois.step()


0 4.513723373413086 0.5056179775280899 0.391304347826087
1 4.472173690795898 0.5056179775280899 0.391304347826087
2 4.36558723449707 0.4157303370786517 0.391304347826087
3 4.43808650970459 0.43820224719101125 0.391304347826087
4 4.496966361999512 0.5056179775280899 0.391304347826087
5 4.431865215301514 0.43820224719101125 0.391304347826087
6 4.399778842926025 0.4943820224719101 0.391304347826087
7 4.539257526397705 0.42696629213483145 0.391304347826087
8 4.487360000610352 0.47191011235955055 0.391304347826087
9 4.424665927886963 0.42696629213483145 0.391304347826087
10 4.37591028213501 0.449438202247191 0.391304347826087
11 4.60662841796875 0.550561797752809 0.391304347826087
12 4.624963283538818 0.4943820224719101 0.391304347826087
13 4.293585777282715 0.5393258426966292 0.391304347826087
14 4.494107723236084 0.42696629213483145 0.391304347826087
15 4.433464527130127 0.5280898876404494 0.391304347826087
16 4.360749244689941 0.47191011235955055 0.391304347826087
17 4.468668460845947 0.

141 4.4250874519348145 0.47191011235955055 0.391304347826087
142 4.473495006561279 0.4606741573033708 0.391304347826087
143 4.29237174987793 0.47191011235955055 0.391304347826087
144 4.401815414428711 0.42696629213483145 0.391304347826087
145 4.369826316833496 0.4606741573033708 0.391304347826087
146 4.411806106567383 0.4606741573033708 0.391304347826087
147 4.523090362548828 0.5280898876404494 0.391304347826087
148 4.510478496551514 0.4943820224719101 0.391304347826087
149 4.482841968536377 0.47191011235955055 0.391304347826087
150 4.314613342285156 0.4943820224719101 0.391304347826087
151 4.3772053718566895 0.449438202247191 0.391304347826087
152 4.507637023925781 0.5168539325842697 0.391304347826087
153 4.351282119750977 0.43820224719101125 0.391304347826087
154 4.298178195953369 0.5280898876404494 0.391304347826087
155 4.436042785644531 0.47191011235955055 0.391304347826087
156 4.399289131164551 0.48314606741573035 0.391304347826087
157 4.60286808013916 0.449438202247191 0.39130434

In [38]:
neural_pois.eval()
mu=neural_pois(X_te).data.numpy()

In [40]:
probs=util.calc_pois_probs(mu[:,0],mu[:,1])
util.multi_result(y=y_test,y_prob=probs,top_n=5,verbose=True)

wrong prediction for [2 1]
candidates 
 [[1.         1.         0.12700998]
 [0.         1.         0.10558353]
 [1.         0.         0.09535259]
 [1.         2.         0.08458886]
 [0.         0.         0.07926671]]
right prediction for [1 0]
candidates 
 [[1.         1.         0.11458894]
 [1.         2.         0.0874247 ]
 [0.         1.         0.08193821]
 [2.         1.         0.08012516]
 [1.         0.         0.07509677]]
wrong prediction for [1 3]
candidates 
 [[1.         1.         0.10815176]
 [2.         1.         0.08889735]
 [1.         2.         0.07822489]
 [1.         0.         0.07476395]
 [0.         1.         0.06578825]]
right prediction for [2 1]
candidates 
 [[1.         1.         0.11944042]
 [0.         1.         0.09021496]
 [1.         2.         0.08719963]
 [1.         0.         0.08180088]
 [2.         1.         0.07906679]]
wrong prediction for [2 0]
candidates 
 [[1.         1.         0.11366611]
 [1.         2.         0.08491191]
 [2.

0.391304347826087

### Results
Poisson regression suffers from the same problem as negative binomial regression with fixed dispersion $\alpha$ (it seems to memorize the most frequent outcomes) 

## Group Stage

In [41]:
data_train, data_test = train_test_split(data_gr.copy(),test_size=0.2)

In [42]:
teamA_def_mean = data_train["teamA_def_val"].mean()
teamA_def_std = data_train["teamA_def_val"].std()
teamA_off_mean = data_train["teamA_off_val"].mean()
teamA_off_std = data_train["teamA_off_val"].std()

teamB_def_mean = data_train["teamB_def_val"].mean()
teamB_def_std = data_train["teamB_def_val"].std()
teamB_off_mean = data_train["teamB_off_val"].mean()
teamB_off_std = data_train["teamB_off_val"].std()

teamA_frag_mean = data_train["teamA_frag"].mean()
teamA_frag_std = data_train["teamA_frag"].std()

teamB_frag_mean = data_train["teamB_frag"].mean()
teamB_frag_std = data_train["teamB_frag"].std()

teamA_age_mean = data_train["teamA_age"].mean()
teamA_age_std = data_train["teamA_age"].std()

teamB_age_mean = data_train["teamB_age"].mean()
teamB_age_std = data_train["teamB_age"].std()

In [43]:
# scale some features
data_train["teamA_def_val"]=(data_train["teamA_def_val"]-teamA_def_mean)/teamA_def_std
data_train["teamA_off_val"]=(data_train["teamA_off_val"]-teamA_off_mean)/teamA_off_std
data_train["teamB_def_val"]=(data_train["teamB_def_val"]-teamB_def_mean)/teamB_def_std
data_train["teamB_off_val"]=(data_train["teamB_off_val"]-teamB_off_mean)/teamB_off_std
data_train["teamA_frag"]=(data_train["teamA_frag"]-teamA_frag_mean)/teamA_frag_std
data_train["teamB_frag"]=(data_train["teamB_frag"]-teamB_frag_mean)/teamB_frag_std
data_train["teamA_age"]=(data_train["teamA_age"]-teamA_age_mean)/teamA_age_std
data_train["teamB_age"]=(data_train["teamB_age"]-teamB_age_mean)/teamB_age_std

# scale test features
data_test["teamA_def_val"]=(data_test["teamA_def_val"]-teamA_def_mean)/teamA_def_std
data_test["teamA_off_val"]=(data_test["teamA_off_val"]-teamA_off_mean)/teamA_off_std
data_test["teamB_def_val"]=(data_test["teamB_def_val"]-teamB_def_mean)/teamB_def_std
data_test["teamB_off_val"]=(data_test["teamB_off_val"]-teamB_off_mean)/teamB_off_std
data_test["teamA_frag"]=(data_test["teamA_frag"]-teamA_frag_mean)/teamA_frag_std
data_test["teamB_frag"]=(data_test["teamB_frag"]-teamB_frag_mean)/teamB_frag_std
data_test["teamA_age"]=(data_test["teamA_age"]-teamA_age_mean)/teamA_age_std
data_test["teamB_age"]=(data_test["teamB_age"]-teamB_age_mean)/teamB_age_std

In [44]:
data_train.describe()

,gametype,resultA,resultB,teamA_age,teamB_age,teamA_def_val,teamB_def_val,teamA_off_val,teamB_off_val,teamA_frag,teamB_frag,past_resultA,past_resultB
count,335.0,335.000000,335.000000,3.350000e+02,3.350000e+02,3.350000e+02,3.350000e+02,3.350000e+02,3.350000e+02,3.350000e+02,3.350000e+02,335.000000,335.000000
mean,6.0,1.286567,1.086567,1.320337e-15,9.968809e-16,-8.948066e-18,-1.378665e-16,2.651279e-18,-2.121023e-17,1.007486e-16,1.643793e-16,0.932388,0.862776
std,0.0,1.238709,1.108608,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.138333,1.183526
min,6.0,0.000000,0.000000,-3.235472e+00,-3.528631e+00,-8.703205e-01,-7.889466e-01,-8.711014e-01,-8.174861e-01,-2.995404e+00,-2.334347e+00,0.000000,0.000000
25%,6.0,0.000000,0.000000,-6.323328e-01,-5.730365e-01,-7.064595e-01,-6.547845e-01,-7.106865e-01,-6.816657e-01,-5.853464e-01,-7.101303e-01,0.000000,0.000000
50%,6.0,1.000000,1.000000,-1.558895e-02,7.048935e-02,-3.413839e-01,-3.774790e-01,-4.006637e-01,-3.833653e-01,7.194202e-02,2.643999e-01,0.950000,0.050000
75%,6.0,2.000000,2.000000,7.613481e-01,6.330928e-01,2.871698e-01,1.106148e-01,3.749480e-01,2.347915e-01,9.483266e-01,9.140867e-01,1.900000,1.120000
max,6.0,7.000000,5.000000,2.283184e+00,2.243834e+00,3.993425e+00,4.756678e+00,3.481151e+00,4.638255e+00,1.605615e+00,1.563774e+00,4.900000,5.700000


In [45]:
col=["teamA_age","teamB_age","teamA_def_val","teamB_def_val","teamA_off_val","teamB_off_val","teamA_frag","teamB_frag","past_resultA","past_resultB"]

X_train = data_train[col].values
y_train = data_train[["resultA","resultB"]].values
X_test = data_test[col].values
y_test = data_test[["resultA","resultB"]].values

### Negative Binomial Regression

In [46]:
X_train = data_train[col].values
y_train = data_train[["resultA","resultB"]].values
X_test = data_test[col].values
y_test = data_test[["resultA","resultB"]].values

X_tr = torch.from_numpy(X_train).float()
y_trA = torch.from_numpy(y_train[:,[0]]).float()
y_trB = torch.from_numpy(y_train[:,[1]]).float()
X_te = torch.from_numpy(X_test).float()

In [47]:
crit = NBNLLLoss(eps=1e-4,verbose=False)
neural = NBNet(len(col),30,20,4,0.3)
optimizer = optim.RMSprop(params=neural.parameters(), lr=1e-3, alpha=0.99, eps=1e-05, weight_decay=0, momentum=0, centered=False)

In [48]:
for epoch in range(200):
    # train
    neural.train()
    muA,alphaA,muB,alphaB = neural(X_tr)
    lossA = crit(muA,alphaA,y_trA)
    lossB = crit(muB,alphaB,y_trB)
    loss = lossA+lossB
    muA,alphaA,muB,alphaB=to_numpy(muA,alphaA,muB,alphaB)
    tend_acc_tr = util.tend_acc_nb(muA,alphaA,muB,alphaB,y_train)
    # evaluate
    neural.eval()
    muA,alphaA,muB,alphaB = neural(X_te)
    muA,alphaA,muB,alphaB=to_numpy(muA,alphaA,muB,alphaB)
    tend_acc_val = util.tend_acc_nb(muA,alphaA,muB,alphaB,y_test)
    # update
    print(epoch,loss.item(),tend_acc_tr,tend_acc_val)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 3.02915096282959 0.35522388059701493 0.3333333333333333
1 2.984318971633911 0.3582089552238806 0.40476190476190477
2 2.967223644256592 0.34328358208955223 0.39285714285714285
3 2.9429664611816406 0.382089552238806 0.40476190476190477
4 2.93709659576416 0.39104477611940297 0.40476190476190477
5 2.9223203659057617 0.4298507462686567 0.4166666666666667
6 2.9139623641967773 0.4298507462686567 0.4166666666666667
7 2.9042115211486816 0.43582089552238806 0.4166666666666667
8 2.9101028442382812 0.43283582089552236 0.40476190476190477
9 2.8892102241516113 0.43582089552238806 0.42857142857142855
10 2.9018993377685547 0.44477611940298506 0.42857142857142855
11 2.882643699645996 0.4388059701492537 0.42857142857142855
12 2.888030529022217 0.46567164179104475 0.42857142857142855
13 2.8870251178741455 0.4537313432835821 0.42857142857142855
14 2.8747756481170654 0.4388059701492537 0.44047619047619047
15 2.8738889694213867 0.46567164179104475 0.44047619047619047
16 2.8619143962860107 0.47462686567164

137 2.692866802215576 0.5194029850746269 0.5238095238095238
138 2.7241668701171875 0.5164179104477612 0.5119047619047619
139 2.6921772956848145 0.5134328358208955 0.5119047619047619
140 2.7062034606933594 0.5373134328358209 0.5238095238095238
141 2.7050576210021973 0.5373134328358209 0.5357142857142857
142 2.6868677139282227 0.5402985074626866 0.5119047619047619
143 2.697754144668579 0.5223880597014925 0.5119047619047619
144 2.7013378143310547 0.5432835820895522 0.5119047619047619
145 2.6843247413635254 0.5343283582089552 0.5119047619047619
146 2.7088630199432373 0.5164179104477612 0.5238095238095238
147 2.6830239295959473 0.5313432835820896 0.5238095238095238
148 2.7178354263305664 0.5283582089552239 0.5238095238095238
149 2.692103147506714 0.5223880597014925 0.5357142857142857
150 2.698939561843872 0.5223880597014925 0.5357142857142857
151 2.6996564865112305 0.5104477611940299 0.5119047619047619
152 2.685128927230835 0.5283582089552239 0.5119047619047619
153 2.6946287155151367 0.5402

In [49]:
neural.eval()
muA,alphaA,muB,alphaB=neural(X_te)
muA,alphaA,muB,alphaB=to_numpy(muA,alphaA,muB,alphaB)

In [52]:
probs = util.calc_nb_probs(muA,alphaA,muB,alphaB)
util.multi_result(y=y_test,y_prob=probs,top_n=1,verbose=True)

wrong prediction for [0 1]
candidates 
 [[1.         0.         0.14152381]]
wrong prediction for [3 0]
candidates 
 [[1.         0.         0.12850194]]
right prediction for [1 0]
candidates 
 [[1.         0.         0.15107851]]
wrong prediction for [3 1]
candidates 
 [[1.         0.         0.14418252]]
wrong prediction for [2 2]
candidates 
 [[1.         0.         0.16262281]]
wrong prediction for [1 0]
candidates 
 [[0.         1.         0.12069098]]
wrong prediction for [2 1]
candidates 
 [[1.         0.         0.16835176]]
wrong prediction for [1 1]
candidates 
 [[1.         0.         0.13468313]]
wrong prediction for [2 0]
candidates 
 [[1.         0.         0.18738873]]
wrong prediction for [1 1]
candidates 
 [[0.         0.         0.14478494]]
wrong prediction for [3 0]
candidates 
 [[1.         0.         0.16787467]]
right prediction for [1 0]
candidates 
 [[1.         0.         0.13282682]]
right prediction for [0 1]
candidates 
 [[0.         1.         0.16680049]]

0.13095238095238096

In [53]:
state = {
    "teamA_def_mean": teamA_def_mean,
    "teamA_def_std": teamA_def_std,
    "teamA_off_mean": teamA_off_mean,
    "teamA_off_std": teamA_off_std,
    "teamB_def_mean": teamB_def_mean,
    "teamB_def_std": teamB_def_std,
    "teamB_off_mean": teamB_off_mean,
    "teamB_off_std": teamB_off_std,
    "teamA_frag_mean": teamA_frag_mean, 
    "teamA_frag_std": teamA_frag_std,
    "teamB_frag_mean": teamB_frag_mean, 
    "teamB_frag_std": teamB_frag_std,
    "teamA_age_mean": teamA_age_mean, 
    "teamA_age_std": teamA_age_std,
    "teamB_age_mean": teamB_age_mean,
    "teamB_age_std": teamB_age_std,
    "state_dict": neural.state_dict(),
}
#torch.save(state, 'model/model_r3.pth')

### Poisson Regression

In [55]:
crit_pois = torch.nn.PoissonNLLLoss()
neural_pois = PoNet(len(col),30,20,2,0.3)
optimizer_pois = optim.RMSprop(params=neural.parameters(), lr=1e-3, alpha=0.99, eps=1e-05, weight_decay=0, momentum=0, centered=False)

In [56]:
for epoch in range(200):
    # train
    neural_pois.train()
    mu = neural_pois(X_tr)
    lossA_pois = crit_pois(mu[:,[0]],y_trA)
    lossB_pois = crit_pois(mu[:,[1]],y_trB)
    loss_pois = lossA_pois+lossB_pois    
    tend_acc_tr = util.tend_acc_pois(mu.data.numpy(),y_train)
    # evaluate
    neural_pois.eval()
    mu = neural_pois(X_te)
    tend_acc_val = util.tend_acc_pois(mu.data.numpy(),y_test)
    print(epoch, loss_pois.item(),tend_acc_tr,tend_acc_val)
    # update
    optimizer_pois.zero_grad()
    loss_pois.backward()
    optimizer_pois.step()

0 3.822608470916748 0.29253731343283584 0.30952380952380953
1 3.909024238586426 0.3074626865671642 0.30952380952380953
2 3.7601242065429688 0.31044776119402984 0.30952380952380953
3 3.8498096466064453 0.31044776119402984 0.30952380952380953
4 3.886183500289917 0.2865671641791045 0.30952380952380953
5 3.9369940757751465 0.2656716417910448 0.30952380952380953
6 3.8860971927642822 0.3164179104477612 0.30952380952380953
7 3.843522310256958 0.31343283582089554 0.30952380952380953
8 3.880828619003296 0.3194029850746269 0.30952380952380953
9 3.9333205223083496 0.3283582089552239 0.30952380952380953
10 3.8387069702148438 0.3283582089552239 0.30952380952380953
11 3.8445682525634766 0.32238805970149254 0.30952380952380953
12 3.8278656005859375 0.2955223880597015 0.30952380952380953
13 3.7638604640960693 0.3164179104477612 0.30952380952380953
14 3.8459274768829346 0.31044776119402984 0.30952380952380953
15 3.8478012084960938 0.28955223880597014 0.30952380952380953
16 3.8528056144714355 0.33134328

134 3.7900476455688477 0.3074626865671642 0.30952380952380953
135 3.901623249053955 0.29850746268656714 0.30952380952380953
136 3.9296488761901855 0.31044776119402984 0.30952380952380953
137 3.9260127544403076 0.3194029850746269 0.30952380952380953
138 3.815800666809082 0.3373134328358209 0.30952380952380953
139 3.821709156036377 0.33134328358208953 0.30952380952380953
140 3.810063362121582 0.29253731343283584 0.30952380952380953
141 3.8914575576782227 0.3164179104477612 0.30952380952380953
142 3.8823795318603516 0.28955223880597014 0.30952380952380953
143 3.8600616455078125 0.28955223880597014 0.30952380952380953
144 3.8364827632904053 0.31343283582089554 0.30952380952380953
145 3.916614294052124 0.29850746268656714 0.30952380952380953
146 3.7954201698303223 0.29850746268656714 0.30952380952380953
147 3.9195587635040283 0.2835820895522388 0.30952380952380953
148 3.812534809112549 0.2955223880597015 0.30952380952380953
149 3.9058456420898438 0.33432835820895523 0.30952380952380953
150 

In [57]:
neural_pois.eval()
mu=neural_pois(X_te).data.numpy()

In [58]:
probs=util.calc_pois_probs(mu[:,0],mu[:,1])
util.multi_result(y=y_test,y_prob=probs,top_n=5,verbose=True)

right prediction for [0 1]
candidates 
 [[1.         1.         0.13505781]
 [1.         0.         0.13061192]
 [0.         1.         0.12795515]
 [0.         0.         0.12374307]
 [2.         1.         0.07127736]]
wrong prediction for [3 0]
candidates 
 [[1.         1.         0.13296953]
 [0.         1.         0.12389869]
 [1.         0.         0.11231888]
 [0.         0.         0.10465677]
 [1.         2.         0.07870848]]
right prediction for [1 0]
candidates 
 [[1.         1.         0.13043708]
 [0.         1.         0.11559934]
 [1.         0.         0.10337009]
 [0.         0.         0.09161133]
 [1.         2.         0.08229572]]
wrong prediction for [3 1]
candidates 
 [[1.         1.         0.13254125]
 [1.         0.         0.11695304]
 [0.         1.         0.11348409]
 [0.         0.         0.10013721]
 [2.         1.         0.07739932]]
wrong prediction for [2 2]
candidates 
 [[1.         1.         0.13331675]
 [0.         1.         0.12227116]
 [1.

0.5357142857142857

# Inference

In [ ]:
#https://pytorch.org/docs/master/notes/serialization.html#best-practices
#https://stackoverflow.com/questions/42703500/best-way-to-save-a-trained-model-in-pytorch

### Pickled

In [59]:
col=["teamA_age","teamB_age","teamA_def_val","teamB_def_val","teamA_off_val","teamB_off_val","teamA_frag","teamB_frag","past_resultA","past_resultB"]

#### Game 1

In [63]:
# 1. Spieltag
neural_pickled = NBNet(len(col),20,20,4,0.2)
state = torch.load('model/model_r1.pth')
neural_pickled.load_state_dict(state["state_dict"])

In [64]:
wm = pd.read_csv(filepath_or_buffer="data/WM18.csv",delimiter=";",index_col=False).round(2)
# impute missing past values with 0
wm.fillna({"past_resultA":0,"past_resultB":0},inplace=True)
#scale
wm["teamA_def_val"]=(wm["teamA_def_val"]-state["teamA_def_mean"])/state["teamA_def_std"]
wm["teamA_off_val"]=(wm["teamA_off_val"]-state["teamA_off_mean"])/state["teamA_off_std"]
wm["teamB_def_val"]=(wm["teamB_def_val"]-state["teamB_def_mean"])/state["teamB_def_std"]
wm["teamB_off_val"]=(wm["teamB_off_val"]-state["teamB_off_mean"])/state["teamB_off_std"]
wm["teamA_frag"]=(wm["teamA_frag"]-state["teamA_frag_mean"])/state["teamA_frag_std"]
wm["teamB_frag"]=(wm["teamB_frag"]-state["teamB_frag_mean"])/state["teamB_frag_std"]
wm["teamA_age"]=(wm["teamA_age"]-state["teamA_age_mean"])/state["teamA_age_std"]
wm["teamB_age"]=(wm["teamB_age"]-state["teamB_age_mean"])/state["teamB_age_std"]
wm[0:16]

,gameid,tournament,gametype,teamA,teamidA,teamB,teamidB,resultA,resultB,addinfo,...,teamA_age,teamB_age,teamA_def_val,teamB_def_val,teamA_off_val,teamB_off_val,teamA_frag,teamB_frag,past_resultA,past_resultB
0,2977683,WM18,Gruppe A,Russland,3448,Saudi-Arabien,3807,5,0,NaN,...,1.420402,1.341508,0.112267,-0.636883,0.505582,-0.708195,-1.221270,-1.887689,0.00,0.00
1,2977684,WM18,Gruppe A,Ägypten,3672,Uruguay,3449,0,1,NaN,...,1.557573,1.001132,-0.333798,1.596695,1.250698,1.949058,0.082092,0.699423,0.00,1.90
2,2977690,WM18,Gruppe B,Portugal,3300,Spanien,3375,3,3,NaN,...,1.137992,1.238603,1.889089,7.625774,3.335494,5.612593,0.733772,-0.594133,2.09,3.80
3,2977689,WM18,Gruppe B,Marokko,3575,Iran,3582,0,1,NaN,...,0.226210,0.169981,-0.134185,-0.581801,0.281588,-0.397746,1.385453,0.483830,0.00,0.00
4,2977696,WM18,Gruppe C,Peru,3584,Dänemark,3436,0,1,NaN,...,0.371449,0.241222,-0.663423,0.811036,-0.461386,1.143794,0.733772,0.699423,0.00,0.00
5,2977695,WM18,Gruppe C,Frankreich,3377,Australien,3433,2,1,NaN,...,-0.750123,0.929890,6.658797,-0.427231,7.173991,-0.480413,-0.135135,1.346201,5.70,0.00
6,2977702,WM18,Gruppe D,Kroatien,3556,Nigeria,3444,2,0,NaN,...,0.750686,-0.795737,1.064370,0.056974,2.314670,0.188658,1.168226,1.130608,0.00,0.00
7,2977701,WM18,Gruppe D,Argentinien,3437,Island,3574,1,1,NaN,...,2.090118,1.373171,1.195695,-0.574970,6.323304,-0.070645,0.516545,1.130608,0.00,0.00
8,2977708,WM18,Gruppe E,Brasilien,3439,Schweiz,3384,1,1,NaN,...,1.315507,0.169981,5.476876,1.717959,6.782308,0.022133,-0.352362,0.699423,0.10,1.00
9,2977707,WM18,Gruppe E,Costa Rica,8497,Serbien,3438,0,1,NaN,...,2.299909,-0.138732,-0.328545,0.929738,-0.675588,1.229435,0.733772,1.346201,0.00,0.00


In [65]:
wm_tr = torch.from_numpy(wm[0:16][col].values).float()
neural_pickled.eval()
muA,alphaA,muB,alphaB=neural_pickled(wm_tr)
muA,alphaA,muB,alphaB=to_numpy(muA,alphaA,muB,alphaB)
probs = util.calc_nb_probs(muA,alphaA,muB,alphaB)
util.multi_result(y_prob=probs,top_n=1,verbose=True)

candidates 
 [[2.         0.         0.14992233]]
candidates 
 [[0.        1.        0.1486018]]
candidates 
 [[0.         1.         0.18752531]]
candidates 
 [[1.         0.         0.14686722]]
candidates 
 [[0.        1.        0.1537527]]
candidates 
 [[1.         0.         0.20177939]]
candidates 
 [[1.       0.       0.152191]]
candidates 
 [[1.         0.         0.17163099]]
candidates 
 [[2.         0.         0.17726191]]
candidates 
 [[0.         1.         0.16916594]]
candidates 
 [[1.         0.         0.18694237]]
candidates 
 [[1.         1.         0.12095687]]
candidates 
 [[0.         1.         0.21038033]]
candidates 
 [[2.       0.       0.175076]]
candidates 
 [[0.         0.         0.13923326]]
candidates 
 [[1.         1.         0.13094899]]


0.0

Top-5 accuracy of 56.25%

Top-3 accuracy of 31.25%

Top-1 accuracy of 18.75%


In [67]:
util.multi_result(y_prob=probs,y=wm[0:16][["resultA","resultB"]].values,top_n=1,verbose=True)

wrong prediction for [5 0]
candidates 
 [[2.         0.         0.14992233]]
right prediction for [0 1]
candidates 
 [[0.        1.        0.1486018]]
wrong prediction for [3 3]
candidates 
 [[0.         1.         0.18752531]]
wrong prediction for [0 1]
candidates 
 [[1.         0.         0.14686722]]
right prediction for [0 1]
candidates 
 [[0.        1.        0.1537527]]
wrong prediction for [2 1]
candidates 
 [[1.         0.         0.20177939]]
wrong prediction for [2 0]
candidates 
 [[1.       0.       0.152191]]
wrong prediction for [1 1]
candidates 
 [[1.         0.         0.17163099]]
wrong prediction for [1 1]
candidates 
 [[2.         0.         0.17726191]]
right prediction for [0 1]
candidates 
 [[0.         1.         0.16916594]]
wrong prediction for [0 1]
candidates 
 [[1.         0.         0.18694237]]
wrong prediction for [1 0]
candidates 
 [[1.         1.         0.12095687]]
wrong prediction for [1 2]
candidates 
 [[0.         1.         0.21038033]]
wrong predi

0.1875

In [68]:
util.single_tendency(y=wm[0:16][["resultA","resultB"]].values,y_prob=probs)

0.5

In [69]:
util.multi_tendency(y=wm[0:16][["resultA","resultB"]].values,y_prob=probs)

0.5625

#### Game 2

In [71]:
# 2. Spieltag
neural_pickled = NBNet(len(col),30,20,4,0.3)
state = torch.load('model/model_r2.pth')
neural_pickled.load_state_dict(state["state_dict"])

Process World Cup Matches

In [72]:
wm = pd.read_csv(filepath_or_buffer="data/WM18.csv",delimiter=";",index_col=False).round(2)

In [73]:
# impute missing past values with 0
wm.fillna({"past_resultA":0,"past_resultB":0},inplace=True)

In [74]:
#scale
wm["teamA_def_val"]=(wm["teamA_def_val"]-state["teamA_def_mean"])/state["teamA_def_std"]
wm["teamA_off_val"]=(wm["teamA_off_val"]-state["teamA_off_mean"])/state["teamA_off_std"]
wm["teamB_def_val"]=(wm["teamB_def_val"]-state["teamB_def_mean"])/state["teamB_def_std"]
wm["teamB_off_val"]=(wm["teamB_off_val"]-state["teamB_off_mean"])/state["teamB_off_std"]
wm["teamA_frag"]=(wm["teamA_frag"]-state["teamA_frag_mean"])/state["teamA_frag_std"]
wm["teamB_frag"]=(wm["teamB_frag"]-state["teamB_frag_mean"])/state["teamB_frag_std"]
wm["teamA_age"]=(wm["teamA_age"]-state["teamA_age_mean"])/state["teamA_age_std"]
wm["teamB_age"]=(wm["teamB_age"]-state["teamB_age_mean"])/state["teamB_age_std"]

In [75]:
wm[16:32]

,gameid,tournament,gametype,teamA,teamidA,teamB,teamidB,resultA,resultB,addinfo,...,teamA_age,teamB_age,teamA_def_val,teamB_def_val,teamA_off_val,teamB_off_val,teamA_frag,teamB_frag,past_resultA,past_resultB
16,2977686,WM18,Gruppe A,Uruguay,3449,Saudi-Arabien,3807,1.0,0.0,NaN,...,0.922794,1.268223,1.176461,-0.566439,1.286868,-0.683698,0.702228,-1.911225,0.00,0.00
17,2977685,WM18,Gruppe A,Russland,3448,Ägypten,3672,3.0,1.0,NaN,...,1.428565,1.499183,-0.053747,-0.261908,0.045351,1.421670,-1.351346,0.036061,0.00,0.00
18,2977692,WM18,Gruppe B,Iran,3582,Spanien,3375,0.0,1.0,NaN,...,0.066245,1.168140,-0.619397,7.652497,-0.477911,6.288057,0.474053,-0.613035,0.00,0.00
19,2977691,WM18,Gruppe B,Portugal,3300,Marokko,3575,1.0,0.0,NaN,...,1.094103,0.228903,1.374984,-0.068231,2.247523,0.382925,0.702228,1.334252,0.00,0.00
20,2977697,WM18,Gruppe C,Frankreich,3377,Peru,3584,1.0,0.0,NaN,...,-0.814776,0.367479,5.210277,-0.581729,5.052564,-0.413434,-0.210472,0.685156,0.00,0.00
21,2977698,WM18,Gruppe C,Dänemark,3436,Australien,3433,1.0,1.0,NaN,...,0.139664,0.867892,0.528798,-0.357897,0.681315,-0.432458,0.702228,1.334252,0.00,0.00
22,2977703,WM18,Gruppe D,Argentinien,3437,Kroatien,3556,0.0,3.0,NaN,...,2.056700,0.729316,0.817430,1.094677,4.430911,2.562091,0.474053,1.117886,0.00,0.00
23,2977704,WM18,Gruppe D,Nigeria,3444,Island,3574,2.0,0.0,NaN,...,-0.928982,1.299017,-0.092818,-0.504853,-0.036940,0.019513,1.158577,1.117886,0.00,0.00
24,2977710,WM18,Gruppe E,Serbien,3438,Schweiz,3384,1.0,2.0,NaN,...,-0.251901,0.128820,0.626651,-0.740153,0.745717,0.121845,1.386752,0.685156,0.00,0.00
25,2977709,WM18,Gruppe E,Brasilien,3439,Costa Rica,8497,2.0,0.0,NaN,...,1.273570,2.207460,4.259901,-0.256811,4.766335,-0.643027,-0.438646,0.685156,1.01,0.00


In [76]:
wm_tr = torch.from_numpy(wm[16:32][col].values).float()

In [77]:
neural_pickled.eval()
muA,alphaA,muB,alphaB=neural_pickled(wm_tr)
muA,alphaA,muB,alphaB=to_numpy(muA,alphaA,muB,alphaB)
probs = util.calc_nb_probs(muA,alphaA,muB,alphaB)
util.multi_result(y_prob=probs,top_n=1,verbose=True)

candidates 
 [[2.        0.        0.1408494]]
candidates 
 [[0.         1.         0.15445287]]
candidates 
 [[0.         2.         0.15739525]]
candidates 
 [[1.        0.        0.1586748]]
candidates 
 [[1.         0.         0.18686738]]
candidates 
 [[1.         0.         0.15315448]]
candidates 
 [[0.         0.         0.13424797]]
candidates 
 [[1.         0.         0.14404709]]
candidates 
 [[1.         0.         0.14479188]]
candidates 
 [[1.         0.         0.19961584]]
candidates 
 [[0.       0.       0.168282]]
candidates 
 [[1.         1.         0.12689939]]
candidates 
 [[1.         0.         0.18026515]]
candidates 
 [[1.        0.        0.1615339]]
candidates 
 [[0.         0.         0.13958506]]
candidates 
 [[0.         1.         0.15794784]]


0.0

Top-5 accuracy of 50%

Top-3 accuracy of 37.5%

Top-1 accuracy of 12.50%

In [80]:
util.multi_result(y_prob=probs,y=wm[16:32][["resultA","resultB"]].values,top_n=3,verbose=True)

right prediction for [1. 0.]
candidates 
 [[2.         0.         0.1408494 ]
 [3.         0.         0.12900243]
 [1.         0.         0.10569544]]
wrong prediction for [3. 1.]
candidates 
 [[0.         1.         0.15445287]
 [0.         2.         0.12431943]
 [0.         0.         0.11038576]]
right prediction for [0. 1.]
candidates 
 [[0.         2.         0.15739525]
 [0.         3.         0.13457123]
 [0.         1.         0.12376535]]
right prediction for [1. 0.]
candidates 
 [[1.         0.         0.1586748 ]
 [0.         0.         0.13731201]
 [1.         1.         0.12405293]]
right prediction for [1. 0.]
candidates 
 [[1.         0.         0.18686738]
 [2.         0.         0.15924968]
 [0.         0.         0.10997625]]
right prediction for [1. 1.]
candidates 
 [[1.         0.         0.15315448]
 [1.         1.         0.11749234]
 [0.         0.         0.11254028]]
wrong prediction for [0. 3.]
candidates 
 [[0.         0.         0.13424797]
 [1.         0. 

0.375

In [81]:
util.single_tendency(y=wm[16:32][["resultA","resultB"]].values,y_prob=probs)

0.5

In [82]:
util.multi_tendency(y=wm[16:32][["resultA","resultB"]].values,y_prob=probs)

0.6875

In [83]:
#wm_tr.requires_grad_()
#muA,alphaA,muB,alphaB=neural_pickled(wm_tr)
#end = muA.mean()
#end.backward()

In [84]:
#wm_tr.grad.mean(dim=0)

#### Game 3

In [86]:
# 3. Spieltag
neural_pickled = NBNet(len(col),30,20,4,0.3)
state = torch.load('model/model_r3.pth')
neural_pickled.load_state_dict(state["state_dict"])

In [87]:
wm = pd.read_csv(filepath_or_buffer="data/WM18.csv",delimiter=";",index_col=False).round(2)
# impute missing past values with 0
wm.fillna({"past_resultA":0,"past_resultB":0},inplace=True)
#scale
wm["teamA_def_val"]=(wm["teamA_def_val"]-state["teamA_def_mean"])/state["teamA_def_std"]
wm["teamA_off_val"]=(wm["teamA_off_val"]-state["teamA_off_mean"])/state["teamA_off_std"]
wm["teamB_def_val"]=(wm["teamB_def_val"]-state["teamB_def_mean"])/state["teamB_def_std"]
wm["teamB_off_val"]=(wm["teamB_off_val"]-state["teamB_off_mean"])/state["teamB_off_std"]
wm["teamA_frag"]=(wm["teamA_frag"]-state["teamA_frag_mean"])/state["teamA_frag_std"]
wm["teamB_frag"]=(wm["teamB_frag"]-state["teamB_frag_mean"])/state["teamB_frag_std"]
wm["teamA_age"]=(wm["teamA_age"]-state["teamA_age_mean"])/state["teamA_age_std"]
wm["teamB_age"]=(wm["teamB_age"]-state["teamB_age_mean"])/state["teamB_age_std"]

In [88]:
wm[32:48]

,gameid,tournament,gametype,teamA,teamidA,teamB,teamidB,resultA,resultB,addinfo,...,teamA_age,teamB_age,teamA_def_val,teamB_def_val,teamA_off_val,teamB_off_val,teamA_frag,teamB_frag,past_resultA,past_resultB
32,2977687,WM18,Gruppe A,Uruguay,3449,Russland,3448,2.0,0.0,NaN,...,0.940840,1.380574,1.048937,0.144698,1.144058,0.372546,0.723213,-1.246889,0.95,0.95
33,2977688,WM18,Gruppe A,Saudi-Arabien,3807,Ägypten,3672,2.0,1.0,NaN,...,1.232466,1.464562,-0.668931,-0.270062,-0.710576,1.302292,-1.933487,0.027735,0.95,1.95
34,2977694,WM18,Gruppe B,Spanien,3375,Marokko,3575,2.0,2.0,NaN,...,1.190805,0.204733,5.685966,-0.084458,3.682281,0.332109,-0.605137,1.302359,0.00,0.00
35,2977693,WM18,Gruppe B,Iran,3582,Portugal,3300,1.0,1.0,NaN,...,0.065963,1.067525,-0.626567,1.796816,-0.481889,3.389396,0.501821,0.665047,0.00,1.90
36,2977700,WM18,Gruppe C,Australien,3433,Peru,3584,0.0,2.0,NaN,...,0.965836,0.342169,-0.507686,-0.576552,-0.543696,-0.411688,1.165996,0.665047,0.00,0.00
37,2977699,WM18,Gruppe C,Dänemark,3436,Frankreich,3377,0.0,0.0,NaN,...,-0.100680,-0.719142,0.438110,6.231770,0.586143,7.232143,0.723213,-0.184702,0.95,2.00
38,2977706,WM18,Gruppe D,Island,3574,Kroatien,3556,1.0,2.0,NaN,...,1.332452,0.701029,-0.621313,1.029979,-0.255262,2.367441,1.165996,1.089922,0.95,0.10
39,2977705,WM18,Gruppe D,Nigeria,3444,Argentinien,3437,1.0,2.0,NaN,...,-0.950561,1.968494,-0.135279,1.152087,-0.075608,6.380514,1.165996,0.452610,3.90,2.05
40,2977712,WM18,Gruppe E,Schweiz,3384,Costa Rica,8497,2.0,2.0,NaN,...,0.065963,2.167013,1.142203,-0.265177,-0.190982,-0.626127,0.723213,0.665047,0.10,0.95
41,2977711,WM18,Gruppe E,Serbien,3438,Brasilien,3439,0.0,2.0,NaN,...,-0.258991,1.311856,0.535974,4.807179,0.645478,6.840026,1.387388,-0.397140,0.00,0.95


In [89]:
wm_tr = torch.from_numpy(wm[32:48][col].values).float()

In [90]:
neural_pickled.eval()
muA,alphaA,muB,alphaB=neural_pickled(wm_tr)
muA,alphaA,muB,alphaB=to_numpy(muA,alphaA,muB,alphaB)
probs = util.calc_nb_probs(muA,alphaA,muB,alphaB)
util.multi_result(y_prob=probs,top_n=1,verbose=True)

candidates 
 [[1.        0.        0.1366892]]
candidates 
 [[0.         2.         0.14351279]]
candidates 
 [[2.         0.         0.17812292]]
candidates 
 [[0.         1.         0.21517396]]
candidates 
 [[1.         1.         0.12842429]]
candidates 
 [[0.         1.         0.30006071]]
candidates 
 [[0.         2.         0.13592741]]
candidates 
 [[1.         2.         0.09098509]]
candidates 
 [[1.         0.         0.15842456]]
candidates 
 [[0.         2.         0.20304446]]
candidates 
 [[0.         1.         0.22965621]]
candidates 
 [[1.         1.         0.12918017]]
candidates 
 [[1.         1.         0.12307567]]
candidates 
 [[1.         1.         0.13048052]]
candidates 
 [[0.         1.         0.12770414]]
candidates 
 [[1.        1.        0.1296417]]


0.0

Top-5 accuracy 62.5%

Top-3 accuracy 43.75%

Top-1 accuracy 18.75%

In [92]:
util.multi_result(y_prob=probs,y=wm[32:48][["resultA","resultB"]].values,top_n=3,verbose=True)

right prediction for [2. 0.]
candidates 
 [[1.         0.         0.1366892 ]
 [1.         1.         0.12168228]
 [2.         0.         0.09857692]]
wrong prediction for [2. 1.]
candidates 
 [[0.         2.         0.14351279]
 [0.         1.         0.14059555]
 [0.         3.         0.09868203]]
wrong prediction for [2. 2.]
candidates 
 [[2.         0.         0.17812292]
 [1.         0.         0.1777686 ]
 [3.         0.         0.11909493]]
wrong prediction for [1. 1.]
candidates 
 [[0.         1.         0.21517396]
 [0.         2.         0.17225352]
 [0.         0.         0.13449624]]
wrong prediction for [0. 2.]
candidates 
 [[1.         1.         0.12842429]
 [1.         0.         0.10680045]
 [0.         1.         0.10622583]]
right prediction for [0. 0.]
candidates 
 [[0.         1.         0.30006071]
 [0.         2.         0.21337062]
 [0.         0.         0.2109876 ]]
wrong prediction for [1. 2.]
candidates 
 [[0.         2.         0.13592741]
 [0.         1. 

0.4375

In [93]:
util.single_tendency(y=wm[32:48][["resultA","resultB"]].values,y_prob=probs)

0.3125

In [94]:
util.multi_tendency(y=wm[32:48][["resultA","resultB"]].values,y_prob=probs)

0.5625

In [ ]:
wm_tr.requires_grad_()
#wm_tr.grad.data.zero_()
muA,alphaA,muB,alphaB=neural_pickled(wm_tr)
expA = muA.mean()
expA.backward()
wm_tr.grad.mean(dim=0)

In [ ]:
wm_tr.grad.data.zero_()
muA,alphaA,muB,alphaB=neural_pickled(wm_tr)
expB = muB.mean()
expB.backward()
wm_tr.grad.mean(dim=0)

In [ ]:
wm_tr.grad.data.zero_()
muA,alphaA,muB,alphaB=neural_pickled(wm_tr)
disB = alphaB.mean()
disB.backward()
wm_tr.grad.mean(dim=0)

In [ ]:
wm_tr.grad.data.zero_()
muA,alphaA,muB,alphaB=neural_pickled(wm_tr)
disA = alphaA.mean()
disA.backward()
wm_tr.grad.mean(dim=0)

## Knockout Stage

In [95]:
col=["gametype","teamA_age","teamB_age","teamA_def_val","teamB_def_val","teamA_off_val","teamB_off_val","teamA_frag","teamB_frag","past_resultA","past_resultB"]

In [97]:
# Achtelfinale
#neural_pickled = NBNet(len(col),30,30,4,0.25)
#state = torch.load('model/model_ko16.pth')
# Viertelfinale
neural_pickled = NBNet(len(col),30,20,4,0.25)
state = torch.load('model/model_ko8.pth')
neural_pickled.load_state_dict(state["state_dict"])

In [98]:
wm = pd.read_csv(filepath_or_buffer="data/WM18.csv",delimiter=";",index_col=False).round(2)
# impute missing past values with 0
wm.fillna({"past_resultA":0,"past_resultB":0},inplace=True)
#scale
wm["teamA_def_val"]=(wm["teamA_def_val"]-state["teamA_def_mean"])/state["teamA_def_std"]
wm["teamA_off_val"]=(wm["teamA_off_val"]-state["teamA_off_mean"])/state["teamA_off_std"]
wm["teamB_def_val"]=(wm["teamB_def_val"]-state["teamB_def_mean"])/state["teamB_def_std"]
wm["teamB_off_val"]=(wm["teamB_off_val"]-state["teamB_off_mean"])/state["teamB_off_std"]
wm["teamA_frag"]=(wm["teamA_frag"]-state["teamA_frag_mean"])/state["teamA_frag_std"]
wm["teamB_frag"]=(wm["teamB_frag"]-state["teamB_frag_mean"])/state["teamB_frag_std"]
wm["teamA_age"]=(wm["teamA_age"]-state["teamA_age_mean"])/state["teamA_age_std"]
wm["teamB_age"]=(wm["teamB_age"]-state["teamB_age_mean"])/state["teamB_age_std"]

In [99]:
# Encode gametype
rounds = {
    "Finale": 1,
    "Spiel um Platz Drei": 2,
    "Halbfinale": 3,
    "Viertelfinale": 4,
    "Achtelfinale": 5,
    "Gruppenphase": 6
}
def map_to_round(x):
    if x.startswith("Gruppe"):
        return rounds["Gruppenphase"]
    else:
        return rounds[x]
wm["gametype"]=wm["gametype"].apply(map_to_round)

# Achtelfinale
#wm_tr = torch.from_numpy(wm[48:56][col].values).float()
# Viertelfinale
wm_tr = torch.from_numpy(wm[56:60][col].values).float()

In [100]:
neural_pickled.eval()
muA,alphaA,muB,alphaB=neural_pickled(wm_tr)
muA,alphaA,muB,alphaB=to_numpy(muA,alphaA,muB,alphaB)
probs = util.calc_nb_probs(muA,alphaA,muB,alphaB)
util.multi_result(y_prob=probs,top_n=3,verbose=True)

candidates 
 [[1.         0.         0.11342321]
 [1.         1.         0.104496  ]
 [0.         0.         0.08822611]]
candidates 
 [[0.         1.         0.14685746]
 [0.         0.         0.12976716]
 [0.         2.         0.11458328]]
candidates 
 [[0.         0.         0.12782252]
 [0.         1.         0.11566763]
 [1.         0.         0.11556682]]
candidates 
 [[0.         1.         0.1436295 ]
 [0.         0.         0.12048139]
 [0.         2.         0.11516628]]


0.0

In [104]:
#util.single_tendency(y=wm[48:56][["resultA","resultB"]].values,y_prob=probs)
util.single_tendency(y=wm[56:60][["resultA","resultB"]].values,y_prob=probs)

0.5

In [105]:
util.multi_tendency(y=wm[56:60][["resultA","resultB"]].values,y_prob=probs)

0.75

In [106]:
wm[56:60]

,gameid,tournament,gametype,teamA,teamidA,teamB,teamidB,resultA,resultB,addinfo,...,teamA_age,teamB_age,teamA_def_val,teamB_def_val,teamA_off_val,teamB_off_val,teamA_frag,teamB_frag,past_resultA,past_resultB
56,3062384,WM18,4,Brasilien,3439,Belgien,3382,1.0,2.0,NaN,...,1.323929,0.529715,2.773791,1.833333,3.733932,3.936808,-0.833325,0.588942,0.00,0.00
57,3061407,WM18,4,Uruguay,3449,Frankreich,3377,0.0,2.0,NaN,...,0.725801,-0.832652,0.762350,4.990100,0.409147,4.501121,0.586074,-0.298047,0.95,0.00
58,3061408,WM18,4,Russland,3448,Kroatien,3556,NaN,NaN,NaN,...,1.089528,0.672726,-0.342994,0.614451,-0.135983,1.243495,-1.684964,1.254183,0.95,2.85
59,3062884,WM18,4,Schweden,3557,England,3299,0.0,2.0,NaN,...,0.879375,-0.644480,-0.252858,3.750675,-0.544408,3.765804,0.869954,-1.185036,3.90,2.05
